# 7.1 Voting(投票)

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.ensemble import VotingClassifier

In [24]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=200, noise=0.3)

In [3]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)
voting_clf.fit(X,y)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [26]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.4)
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8375
RandomForestClassifier 0.8875
SVC 0.85
VotingClassifier 0.85


# 7.2 Baging
1. 当将数据集的随机子集绘制为样本的随机子集时，该算法称为Pasting[1]。
2. 如果抽取样本进行替换，则该方法称为Bagging [2]。
3. 当将数据集的随机子集绘制为要素的随机子集时，该方法称为Random Subspaces[3]。
4. 最后，当基于样本和特征的子集建立基本估计器时，该方法称为Random Patches[4]。
n_estimators 集成器中基估计器的数量。
max_features 从 X 抽取的样本数以训练每个基本估计器

In [32]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=50 
)
#bootstrap 进行替换是否放回抽样
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.8625

## 7.2.2 OBB（包外）评估
obb_score_ 方法

In [34]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), bootstrap=True, oob_score=True
)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:640: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:644: RuntimeWarning: invalid value encountered in true_divide
  oob_decision_function = (predictions /


0.8583333333333333

# 7.4 随机森林

In [35]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier()
rnd_clf.fit(X_train, y_train)
y_pred = rnd_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.8875

## 7.4.2 极端随机树

In [37]:
from sklearn.ensemble import ExtraTreesClassifier
ex_clf = ExtraTreesClassifier()
ex_clf.fit(X_train, y_train)
y_pred = ex_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.8375

## 7.4.3 特征重要性

In [38]:
ex_clf.feature_importances_

array([0.46042168, 0.53957832])

In [39]:
rnd_clf.feature_importances_

array([0.4562656, 0.5437344])

# 7.5 BOOSTING
## 7.5.1 Adaboost

In [42]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1)
)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1))

In [43]:
y_pred = ada_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.85